In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [12]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [13]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # 배치를 제외한 모든 차원을 평탄화(flatten)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [24]:
device = torch.device('cuda')
net = Net().to(device)

In [25]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [28]:
for epoch in range(10):   # 데이터셋을 수차례 반복합니다.
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        inputs, labels = (t.to(device) for t in data)

        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화를 한 후
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 통계를 출력합니다.
        running_loss += loss.item()
        if i % 400 == 399:    # print every 20 mini-batches
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 400))
            running_loss = 0.0

print('Finished Training')

[1,   400] loss: 1.44643
[1,   800] loss: 1.39325
[1,  1200] loss: 1.38281
[2,   400] loss: 1.35537
[2,   800] loss: 1.34276
[2,  1200] loss: 1.31865
[3,   400] loss: 1.27005
[3,   800] loss: 1.27470
[3,  1200] loss: 1.27500
[4,   400] loss: 1.23224
[4,   800] loss: 1.21464
[4,  1200] loss: 1.19641
[5,   400] loss: 1.17922
[5,   800] loss: 1.16632
[5,  1200] loss: 1.15829
[6,   400] loss: 1.14204
[6,   800] loss: 1.11751
[6,  1200] loss: 1.11495
[7,   400] loss: 1.09391
[7,   800] loss: 1.08094
[7,  1200] loss: 1.08083
[8,   400] loss: 1.06783
[8,   800] loss: 1.04200
[8,  1200] loss: 1.05627
[9,   400] loss: 1.02829
[9,   800] loss: 1.02058
[9,  1200] loss: 1.00481
[10,   400] loss: 0.97621
[10,   800] loss: 0.99756
[10,  1200] loss: 0.98247
Finished Training
